In [ ]:
import redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)
print(r.ping())


ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.

In [ ]:
r.set('foo', 'bar')
# True
r.get('foo')
# bar

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Optional
from datetime import datetime
import json
from enum import Enum

In [ ]:
class WorkingMemorySlot(Enum):
    """
    Cognitive slots in working memory.
    
    Each slot represents a distinct type of information the agent
    might hold in active cognition. The separation allows independent
    access and update—changing the current goal doesn't require
    rewriting the entire working memory.
    """
    CURRENT_GOAL = "current_goal"
    ACTIVE_CONTEXT = "active_context"
    SCRATCHPAD = "scratchpad"
    RECENT_OBSERVATIONS = "recent_observations"
    PENDING_ACTIONS = "pending_actions"
    USER_INTENT = "user_intent"
    REASONING_TRACE = "reasoning_trace"

In [ ]:
@dataclass
class WorkingMemoryEntry:
    """
    A single entry in working memory.
    
    Each entry has content (what we're remembering), metadata (when, how important),
    and lifecycle information (how long until it expires).
    """
    slot: WorkingMemorySlot
    content: Any  # JSON-serializable content
    timestamp: datetime = field(default_factory=datetime.utcnow)
    ttl_seconds: int = 300  # 5 minute default—tune based on your application
    importance: float = 0.5  # 0-1 scale; higher = retain longer under pressure

    def to_redis_hash(self) -> dict:
        """
        Convert to Redis hash format.
        
        Redis hashes store field-value pairs. We serialize complex content
        to JSON since Redis values are strings/bytes. This serialization
        is intentionally simple—if you need more sophisticated serialization,
        consider msgpack or protobuf for better performance.
        """
        return {
            "slot": self.slot.value,
            "content": json.dumps(self.content),
            "timestamp": self.timestamp.isoformat(),
            "importance": str(self.importance)
        }
    
@classmethod
def from_redis_hash(cls, data: dict) -> "WorkingMemoryEntry":
    """
    Reconstruct from Redis hash data.
    
    Note the byte decoding—Redis returns bytes by default.
    """
    return cls(
        slot=WorkingMemorySlot(data[b"slot"].decode()),
        content=json.loads(data[b"content"].decode()),
        timestamp=datetime.fromisoformat(data[b"timestamp"].decode()),
        importance=float(data[b"importance"].decode()),
        ttl_seconds=0  # Already in Redis; TTL managed there
    )

In [ ]:
import json
import redis.asyncio as redis
from datetime import datetime, timezone 
from typing import Any, Optional, List, Dict
from enum import Enum

class WorkingMemorySlot(Enum):
    USER_INTENT = "user_intent"
    CURRENT_GOAL = "current_goal"
    TASK_STATE = "task_state"
    SCRATCHPAD = "scratchpad"

class WorkingMemoryManager:
    def __init__(self, redis_client: redis.Redis, session_id: str, max_observations: int = 10, default_ttl: int = 300):
        self.redis = redis_client
        self.session_id = session_id
        self.max_observations = max_observations
        self.default_ttl = default_ttl
        self._key_prefix = f"wm:{session_id}"

    async def _slot_key(self, slot: WorkingMemorySlot) -> str:
        return f"{self._key_prefix}:{slot.value}"

    async def _safe_execute(self, pipe):
        """Helper to handle both sync and async execute() behaviors."""
        result = pipe.execute()
        import inspect
        if inspect.isawaitable(result):
            return await result
        return result

    async def set_slot(self, slot: WorkingMemorySlot, content: Any, ttl_seconds: Optional[int] = None, importance: float = 0.5) -> None:
        key = await self._slot_key(slot)
        entry_data = {
            "content": json.dumps(content),
            "importance": str(importance),
            "timestamp": datetime.now(timezone.utc).isoformat() 
        }
        pipe = self.redis.pipeline(transaction=True)
        pipe.hset(key, mapping=entry_data)
        pipe.expire(key, ttl_seconds or self.default_ttl)
        await self._safe_execute(pipe) # ✅ Uses safe helper

    async def get_slot(self, slot: WorkingMemorySlot) -> Optional[Dict]:
        key = await self._slot_key(slot)
        # Check if hgetall needs to be awaited
        data_call = self.redis.hgetall(key)
        import inspect
        data = await data_call if inspect.isawaitable(data_call) else data_call
        
        if not data: return None
        decoded = {k.decode() if isinstance(k, bytes) else k: 
                   v.decode() if isinstance(v, bytes) else v 
                   for k, v in data.items()}
        if "content" in decoded:
            decoded["content"] = json.loads(decoded["content"])
        return decoded

    async def append_observation(self, observation: dict) -> None:
        key = f"{self._key_prefix}:observations"
        timestamp = datetime.now(timezone.utc).timestamp() 
        pipe = self.redis.pipeline(transaction=True)
        pipe.zadd(key, {json.dumps(observation): timestamp})
        pipe.zremrangebyrank(key, 0, -(self.max_observations + 1))
        pipe.expire(key, self.default_ttl * 2)
        await self._safe_execute(pipe) # ✅ Uses safe helper

    async def get_recent_observations(self, limit: int = 5) -> List[dict]:
        key = f"{self._key_prefix}:observations"
        obs_call = self.redis.zrange(key, -limit, -1)
        import inspect
        observations = await obs_call if inspect.isawaitable(obs_call) else obs_call
        return [json.loads(obs.decode() if isinstance(obs, bytes) else obs) for obs in observations]

    async def update_scratchpad(self, field: str, value: Any) -> None:
        current = await self.get_slot(WorkingMemorySlot.SCRATCHPAD)
        content = current["content"] if current else {}
        content[field] = value
        await self.set_slot(WorkingMemorySlot.SCRATCHPAD, content)

    async def get_full_context(self) -> dict:
        context = {}
        for slot in WorkingMemorySlot:
            entry = await self.get_slot(slot)
            if entry:
                context[slot.value] = entry["content"]
        context["recent_observations"] = await self.get_recent_observations()
        return context

# --- FIXED Reasoning Function (No Warnings) ---

async def agent_reasoning_step(wm: WorkingMemoryManager, user_input: str, llm: Any) -> str:
    context = await wm.get_full_context()

    await wm.set_slot(
        WorkingMemorySlot.USER_INTENT,
        {
            "raw_input": user_input,
            "parsed_at": datetime.now(timezone.utc).isoformat() # ✅ Fixed Warning
        },
        importance=0.9
    )

    await wm.append_observation({
        "type": "user_input",
        "content": user_input,
        "timestamp": datetime.now(timezone.utc).isoformat() # ✅ Fixed Warning
    })
    
    response = f"Simulated response to: {user_input}"
    await wm.update_scratchpad("last_response", response)
    
    reasoning_steps = context.get("scratchpad", {}).get("reasoning_steps", [])
    reasoning_steps.append({
        "input": user_input,
        "output": response,
        "timestamp": datetime.now(timezone.utc).isoformat() # ✅ Fixed Warning
    })
    await wm.update_scratchpad("reasoning_steps", reasoning_steps)
    
    return response

In [ ]:
import redis.asyncio as redis
import pprint

# Create the client explicitly using the asyncio version
r = redis.Redis(host='localhost', port=6379, decode_responses=False)
wm = WorkingMemoryManager(r, session_id="final_debug_session")

async def test():
    print("🚀 Running reasoning step...")
    await agent_reasoning_step(wm, "Check my memory storage", None)
    
    print("📥 Retrieving context...")
    context = await wm.get_full_context()
    
    print("\n--- AGENT BRAIN OUTPUT ---")
    pprint.pprint(context)

await test()